In [1]:
import cv2

img = cv2.imread("../input/brain-tumor-classification-mri/Testing/meningioma_tumor/image(1).jpg")

img.shape

In [2]:
import matplotlib.pyplot as plt

plt.imshow(img)
plt.show()

In [3]:
train_dir = "../input/brain-tumor-classification-mri/Training"
test_dir = "../input/brain-tumor-classification-mri/Testing"

In [4]:
import tensorflow as tf

train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(224,224),
                                                                 label_mode = "categorical",
                                                                 batch_size = 32
                                                                 )
test_data =tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                               image_size =(224,224),
                                                               label_mode = "categorical",
                                                               batch_size = 32
                                                               )

In [5]:
for image, label in train_data:
  print(image.shape)
  print(label.shape)
  break

In [6]:
label_names = train_data.class_names

label_names, len(label_names)

In [7]:
import matplotlib.pyplot as plt
import tensorflow as tf

plt.figure(figsize=(12,12))

for image, label in train_data.take(1):

  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))
    plt.title(label_names[tf.argmax(label[i])])
    plt.axis("off")


In [84]:
# building a efficientnet model

import tensorflow as tf

base_model = tf.keras.applications.EfficientNetB5(include_top= False)
base_model.trainable = False

In [86]:
# building the model
import tensorflow as tf
from tensorflow.keras import layers

inputs = tf.keras.Input(shape=(224,224,3))

x = base_model(inputs)

x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128, activation= "relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation = "relu")(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(64, activation = "relu")(x)

outputs = layers.Dense(4, activation="softmax")(x)

model = tf.keras.Model(inputs,outputs)


In [87]:
model.summary()

In [88]:
# compiling the model

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)

In [89]:
# fiiting data to the model

model.fit(train_data,
          epochs = 10,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
)

In [90]:
base_model_evluation = model.evaluate(test_data)

In [91]:
print(f"Base Modle Accuracy: {base_model_evluation[1] *100:0.2f} %")

**Tunning the model**

In [92]:
# tunning the model
base_model.trainable = True

for layer in base_model.layers[:-1]:
  base_model.trainable = False

model.compile(
    loss = tf.keras.losses.categorical_crossentropy,
    optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
    metrics = ["accuracy"]
)


In [93]:
model.summary()

In [94]:
# fitting data to the tuned model

model.fit(train_data,
          epochs = 10,
          steps_per_epoch = len(train_data),
          validation_data = test_data,
          validation_steps = len(test_data)
          )

In [95]:
# evaluatinn the model
model_evaluation = model.evaluate(test_data)

In [96]:
print(f"Model Accuracy:{model_evaluation[1] *100: 0.2f} %")

In [97]:
# predicting and visualizing the test data
import tensorflow as tf
import matplotlib.pyplot as plt

plt.figure(figsize=(14,14))

for image, label in test_data.take(1):
  model_prediction = model.predict(image)

  for i in range(9):
    plt.subplot(3,3,i+1)
    plt.imshow(image[i].numpy().astype("uint8"))   
    plt.title(f"Model Prediction :{label_names[tf.argmax(model_prediction[i])]}\n Original : {label_names[tf.argmax(label[i])]}")
